#### Imports/setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import ee
import os
# import pandas as pd
import geemap
import time
import functools
import os
import sys
# from datetime import datetime

import requests # may not be needed
from sidecar import Sidecar

from modules.gee_initialize import initialize_ee 
initialize_ee()

from parameters.config_runtime import (
    geo_id_column, 
    out_csv, 
    out_shapefile,
    cols_ind_1_treecover,
    cols_ind_2_commodities,
    cols_ind_3_dist_before_2020,
    cols_ind_4_dist_after_2020,
)

from modules.agstack_to_gee import (
    start_agstack_session,
    get_agstack_token,
    register_fc_and_append_to_csv,
    # add_geo_ids_to_csv_from_lookup_df,
    add_geo_ids_to_csv_from_lookup_csv,
    add_empty_column_to_csv,
    remove_column_from_csv,
    geo_id_or_ids_to_feature_collection
)

# from modules.stats import get_stats,reformat_whisp_fc,get_stats_formatted

from parameters.config_asr_url_info import asset_registry_base, user_registry_base

from parameters.config_asr_credentials import email, password

from modules.utils import (
    collection_properties_to_df,
    remove_geometry_from_feature_collection,
    get_centroid
)

from modules.tidy_tables import whisp_risk, add_eudr_risk_col, add_indicators, select_years_in_range, create_wildcard_column_list

def whisp_stats_as_df (roi): 
    %store roi
    %run process_whisp_stats.ipynb   
    return df_out
   

#### Set up asset registry

In [ ]:
session = start_agstack_session(email,password,user_registry_base)

token = get_agstack_token(email, password, asset_registry_base)

#### Inputs : without GEOIDS

GEE asset (feature collection) 

In [ ]:
example_plot_feature_col_asset = 'projects/ee-andyarnellgee/assets/p0004_commodity_mapper_support/seg_civ_PCBM'
# example_plot_feature_col_asset = 'projects/ee-cocoacmr/assets/feature_data/cicc_polys_2k'
# Define the fraction of features you want to sample (e.g., 0.5 for 50%)
sampleFraction = 0.0030

In [ ]:
fc = ee.FeatureCollection(example_plot_feature_col_asset)

randomColumn = 'random'
fc_r = fc.randomColumn(randomColumn,4)

# Filter the feature collection based on the random column to get the sample
fc_sample = fc_r.filter(ee.Filter.lt(randomColumn, sampleFraction));

roi = fc_sample

print(roi.size().getInfo())

roi


In [ ]:
# temp code: uncomment below line if want to test using point inputs based on polygon centroid

# roi = roi.map(lambda feature: ee.Feature(feature.geometry().centroid(1))) 

### Whisp it
- If over >500 polygons currently this will go to Google Drive and require manual uploading here
- Stats sent to a separate notebook 'process_whisp_stats.pynb'

In [ ]:
df = whisp_stats_as_df(roi)

### Display it


In [ ]:
df

#### Export it to CSV file
NB skip if exporting to Google Drive

In [ ]:
#export output csv
print  (out_csv)
df.to_csv(path_or_buf=out_csv,header=True,index=False)

print ("output csv: ", out_csv)


#### Add Geo ids
##### Step 1. Register polygons:  compiles geo_ids in a lookup csv
- NB registration takes a long time if many polygons (a few seconds each one). AgStack will update this in future. 
- In the meantime to stop timeouts extend session length click dollar sign and edit under 'Sessions'.
- If processing doesn't complete, rerun and it should pickup from where you got to.

In [ ]:
register_fc_and_append_to_csv(
    feature_col=roi,
    geo_id_column="Geo_id",
    output_lookup_csv="temp_geo_id_lookup.csv",
    join_id_column="system:index",
    token=token,
    session=session,
    asset_registry_base=asset_registry_base,
    override_checks=False,
    remove_temp_csv=False,
    debug=True)


##### Step 2. Join geo ids from lookup csv to Whisp stats csv

In [ ]:
add_geo_ids_to_csv_from_lookup_csv(
        input_csv="whisp_output_table.csv",
        geo_id_lookup_csv="temp_geo_id_lookup.csv",
        join_id_column="system:index",
        geo_id_column=geo_id_column,
        # override_checks=False, # needs implementing
        overwrite=False,
        drop_geo=False,
        debug=True)

##### Optional: remove "system:index" column 
NB this is needed for joining geo_ids to csv (from lookup table). Check you have all your geo_ids first and if in doubt run on a copy of the results


In [ ]:
# remove_column_from_csv(
#     csv_file="copy_whisp_output_table.csv", # this may change depending on if overwrite is on
#     column_name="system:index"
# )

##### Calculate EUDR risk category

In [ ]:
# Replace with the name of your CSV file
csv_file ='copy_whisp_output_table.csv'

# import as a dataframe
df_w_stats = pd.read_csv(csv_file)


df_w_risk = whisp_risk(
    df = df_w_stats, 
    ind_1_pcent_threshold=10,
    ind_2_pcent_threshold=10,
    ind_3_pcent_threshold=0,
    ind_4_pcent_threshold=0
    )

output_file='copy_whisp_output_table_w_risk.csv'

# export to a csv
df_w_risk.to_csv(output_file)


# display df with added risk column
df_w_risk
